# Chapter 116: QSVT

---

**Prerequisites:**
- See `Chapter02_QuantumSoftware.ipynb` for installation instructions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_ibm_runtime import Estimator
from qiskit_aer import Aer
from qiskit.quantum_info import SparsePauliOp
from IPython.display import display
from numpy.polynomial.chebyshev import Chebyshev
from numpy.polynomial import Polynomial
import pyqsp
from pyqsp.angle_sequence import QuantumSignalProcessingPhases
from Chapter17_QSVT_functions import (myQSVT, get_inverse_phases,qsp_response_manual,angles_to_polynomial,
                                      convert_qiskit_to_reflection, 
                                      get_total_error_vs_unscaled_ideal, plot_approximation_comparison)

## SVD

In [ ]:
A = np.array([[0.1, 0.2], [0.2, 0.4]])
U, S, Vh = np.linalg.svd(A,compute_uv=True)

print("Singular values of A:", S)
print("Left singular vectors (U):", U)
print("Right singular vectors (Vh):", Vh)

## SVD always exists

In [ ]:
A = np.array([[2, 1], [0, 2]])
U, S, Vh = np.linalg.svd(A,compute_uv=True)

print("Singular values of A:\n", S)
print("Left singular vectors (U):\n", U)
print("Right singular vectors (Vh):\n", Vh)

print("A ~ \n", U @ np.diag(S) @ Vh)  # Reconstruct A


## From Monomial to Chebyshev

In [2]:
# 1. Define P(x) = -1 + 0*x + 2*x^2 in the standard power basis
# The coefficients are ordered from lowest degree to highest: [a0, a1, a2]
p_coeffs = [-1, 0, 2]
p_standard = Polynomial(p_coeffs)

# 2. Convert to the Chebyshev basis
p_cheb = p_standard.convert(kind=Chebyshev)

# 3. Output the results
print(f"Standard Polynomial: {p_standard}")
print(f"Chebyshev Basis Coefficients: {p_cheb.coef}")

# Interpretation: 
# The output [0. 0. 1.] means 0*T0(x) + 0*T1(x) + 1*T2(x)

Standard Polynomial: -1.0 + 0.0 x + 2.0 x**2
Chebyshev Basis Coefficients: [0. 0. 1.]


## From Chebyshev to Monomials

In [3]:
# 1. Define the coefficients in the Chebyshev basis
# Let's say we have 1*T_0(x) + 0*T_1(x) + 0.5*T_2(x)
cheb_coeffs = [1, 0, 0.5] 
p_cheb = Chebyshev(cheb_coeffs)

# 2. Convert to the Standard Power basis (Polynomial)
p_standard = p_cheb.convert(kind=Polynomial)

# 3. Output the results
print(f"Chebyshev Polynomial: {p_cheb}")
print(f"Standard Basis Coefficients: {p_standard.coef}")

# Interpretation: 
# T_0 is 1, T_2 is (2x^2 - 1). 
# So: 1(1) + 0.5(2x^2 - 1) = 1 + x^2 - 0.5 = 0.5 + x^2
# Result should be [0.5, 0.0, 1.0]

Chebyshev Polynomial: 1.0 + 0.0 T_1(x) + 0.5 T_2(x)
Standard Basis Coefficients: [0.5 0.  1. ]


## QSP: From Angles to Polynomial

In [2]:
angles = [np.pi/2, 0, np.pi/2]
angles_to_polynomial(angles)

Chebyshev([-0.5+6.123234e-17j,  0. +0.000000e+00j,  0. +0.000000e+00j], domain=[-1.,  1.], window=[-1.,  1.], symbol='x')

In [ ]:
# Define polynomial coefficients: P(a) = 2a^2 - 1
# Coefficients in monomial basis: [a^0, a^1, a^2]
from Chapter17_QSVT_functions import convert_qiskit_to_reflection


pcoefs = [-1, 0, 2]

# Convert from monomial to Chebyshev basis (required by pyqsp)
pcoefs_cheb = np.polynomial.chebyshev.poly2cheb(pcoefs)

# Create Chebyshev polynomial object
poly = Chebyshev(pcoefs_cheb)

# Compute QSP phase angles
QiskitPhases = QuantumSignalProcessingPhases(poly, signal_operator="Wz")
reflectionPhases = convert_qiskit_to_reflection(QiskitPhases)

print("Polynomial: P(a) = 2a² - 1")
print([float(phi) for phi in reflectionPhases])



In [ ]:
kappa = 10
eps = 0.01
d = int(3*kappa*np.log(1/eps))

if d % 2 == 0:
    d += 1

print(f"Degree d={d} for kappa={kappa} and eps={eps}")
err = get_total_error_vs_unscaled_ideal(d, kappa)
print(f"Total error vs unscaled ideal: {err}")

## QSVT example

In [ ]:

plot_approximation_comparison(d, kappa)


## Inversion

In [ ]:
A = np.array([[0.5, -0.25], [-0.25, 0.5]])
sigma_vals = np.linalg.svd(A, compute_uv=False)
A = A / (1.001*np.max(sigma_vals))  # Scale to have max singular value = 1
sigma_vals = np.linalg.svd(A, compute_uv=False)
print("Singular values of A:", sigma_vals)
b = np.array([1, 0]) # State |0>
degree = 51
kappa = np.max(sigma_vals) / np.min(sigma_vals)
print(f"Condition number kappa={kappa}")
solver = myQSVT(A, b, degree, kappa, nShots=1000)


u_qsvt = solver.solve()

if u_qsvt is not None:
    print("QSVT Solution (|x>):", u_qsvt)
    
    # Classical Verification
    x_exact = np.linalg.solve(A, b)
    u_exact = x_exact / np.linalg.norm(x_exact)
    print("Classical Solution: ", u_exact)
    print("Fidelity:           ", np.abs(np.dot(u_qsvt.conj(), u_exact))**2)